# Viewing Large Symbolic Outputs in Notebooks

This notebook demonstrates how to use the interactive widget display functionality to view large symbolic expressions that would otherwise be too long to display comfortably in Jupyter/Colab notebooks.

## The Problem

When computing minors and determinants for multi-component FAS systems, the symbolic expressions can grow exponentially large, making them impossible to view or work with in notebooks. Traditional printing would output thousands or millions of characters that freeze the browser.

## The Solution

The `SymbolicExpressionWidget` provides an interactive interface that:
- Shows a compact summary by default (term count, degree, variables)
- Provides expandable sections for different views (LaTeX, terms, etc.)
- Keeps the full SymPy expression accessible via `.expr` for computation
- Works in Jupyter, Colab, and other notebook environments

## Setup

First, make sure you have the required packages installed:

In [ ]:
# Uncomment to install required packages
# !pip install sympy numpy ipywidgets
# !pip install cupy  # Optional, for GPU acceleration

In [ ]:
# Import required modules
import sys
sys.path.insert(0, '..')  # Add parent directory to path

from fas_minor_calculator import FASMinorCalculator
from determinant_computer import DeterminantComputer
from output_display import SymbolicExpressionWidget

import sympy as sp
from IPython.display import display, HTML

## Example 1: Basic Widget Usage

Let's create a simple two-component system and compute a minor using the widget display.

In [ ]:
# Create a two-component system
calc = FASMinorCalculator.from_characteristic_tuples(
    [(3, 1, 5), (3, 1, 4)],  # Two components
    use_symbolic=True
)
det_comp = DeterminantComputer(calc)

print(f"System info:")
print(f"  Components: {len(calc.graphs)}")
print(f"  Total vertices: {calc.total_vertices}")
print(f"  Total edges: {calc.total_edges}")
print(f"  Matrix columns: {calc.total_edges + 1} (edges + b column)")
print(f"  Base rows auto-generated: {len(det_comp.base_rows)}")

### OLD WAY: Direct printing (not recommended for large expressions)

In [ ]:
# WARNING: This may produce very long output!
# Uncomment at your own risk:

# minor = det_comp.compute_minor(0, 0, 1)
# print(minor)  # Could output millions of characters!

### NEW WAY: Using the widget display

In [ ]:
# Compute and display using the widget
widget = det_comp.compute_minor_display(0, 0, 1)

# The widget displays automatically as the last line in a cell
widget

Notice how the widget shows:
- A clean summary with metadata (expression type, size, term count, degree, variables)
- Expandable sections for different views
- The full expression is still accessible but not overwhelming the output

Try clicking on different sections in the accordion to explore!

## Example 2: Accessing the Full Expression

The widget stores the full SymPy expression as `.expr`, so you can still use it for computation:

In [ ]:
# Get the full expression for computation
minor_expr = widget.expr

print(f"Expression type: {type(minor_expr)}")
print(f"Number of terms: {len(minor_expr.args) if isinstance(minor_expr, sp.Add) else 1}")
print(f"Free symbols: {len(minor_expr.free_symbols)}")

# You can do all normal SymPy operations
# Example: substitute a value
symbols = list(minor_expr.free_symbols)
if symbols:
    test_symbol = symbols[0]
    print(f"\nSubstituting {test_symbol} = 0...")
    simplified = minor_expr.subs(test_symbol, 0)
    print(f"Result has {len(str(simplified))} characters")

## Example 3: Viewing Top Terms

You can inspect the first few terms without viewing the entire expression:

In [ ]:
# Show first 5 terms
widget.show_terms(5)

## Example 4: Exporting to File

For very large expressions, you may want to export them to a file for offline viewing or archival:

In [ ]:
# Export to file
filename = widget.export_to_file("my_minor.txt")
print(f"Expression exported to: {filename}")

# Or use a custom filename
custom_filename = widget.export_to_file("minor_component0_vertex0_layer1.txt")
print(f"Also exported to: {custom_filename}")

## Example 5: Computing Multiple Minors

You can compute and compare multiple minors side-by-side:

In [ ]:
# Compute minors for different vertices
print("Computing minors for component 0, layer 1, different vertices:\n")

widget1 = det_comp.compute_minor_display(0, 0, 1)
print("Vertex 0:")
display(widget1)

print("\n" + "="*80 + "\n")

widget2 = det_comp.compute_minor_display(0, 1, 1)
print("Vertex 1:")
display(widget2)

## Example 6: Y-Vector Display

The widget also works with y-vectors computed using Cramer's rule:

In [ ]:
# Compute y-vector as a single widget
y_widget = det_comp.compute_y_vector_display(0, 0, 1)
y_widget

### Y-Vector Components

You can also get individual widgets for each component of the y-vector:

In [ ]:
# Get individual component widgets
y_components = det_comp.compute_y_vector_display(
    0, 0, 1,
    return_mapping=True
)

print(f"Y-vector has {len(y_components)} components (one per edge)\n")

# Display first component
first_edge = list(y_components.keys())[0]
print(f"Component for edge {first_edge}:")
y_components[first_edge]

## Example 7: Customizing Widget Display

You can customize various aspects of the widget:

In [ ]:
# Customize widget parameters
custom_widget = det_comp.compute_minor_display(
    0, 0, 1,
    name="My Custom Minor",           # Custom name
    max_preview_length=1000,          # Longer LaTeX preview
    max_terms_display=20              # Show more terms
)

custom_widget

## Example 8: Direct Widget Creation

You can also create widgets directly from any SymPy expression:

In [ ]:
# Create a large expression for demonstration
x, y, z = sp.symbols('x y z')
big_expr = sp.expand((x + y + z)**12)

# Create widget directly
demo_widget = SymbolicExpressionWidget(
    big_expr,
    name="Expanded (x+y+z)^12"
)

demo_widget

## Example 9: Performance Comparison

Let's compare the performance of different computation methods:

In [ ]:
import time

# Time the fast method
start = time.time()
widget_fast = det_comp.compute_minor_fast_display(0, 0, 1)
fast_time = time.time() - start

print(f"Fast method: {fast_time:.3f} seconds")
print(f"Expression size: {widget_fast.expr_length:,} characters")
print(f"Term count: {widget_fast.term_count:,} terms")

## Example 10: Working with Larger Systems

For larger systems, the widget becomes even more essential:

In [ ]:
# Create a larger system (be patient, this may take a while!)
print("Creating larger system...")
calc_large = FASMinorCalculator.from_characteristic_tuples(
    [(4, 1, 6), (4, 1, 6)],  # Larger components
    use_symbolic=True,
    use_lazy_structure_functions=True,  # Use lazy loading for performance
    enable_simplification=False          # Disable simplification for speed
)
det_comp_large = DeterminantComputer(calc_large)

print(f"System has {calc_large.total_vertices} vertices and {calc_large.total_edges} edges")
print(f"Computing minor... (this may take a minute)")

large_widget = det_comp_large.compute_minor_fast_display(0, 0, 1)
print("\nDone! Widget below:")
large_widget

## Tips and Best Practices

1. **Use `compute_minor_fast_display()`** for better performance on multi-component systems

2. **Export very large expressions** to files instead of trying to view them inline

3. **Use lazy structure functions** (`use_lazy_structure_functions=True`) when creating calculators for large systems

4. **Disable simplification** (`enable_simplification=False`) if you only need the raw expanded form

5. **Access widget.expr** for all computational needs - the widget is just for viewing

6. **Use show_terms(n)** to inspect specific terms without expanding the full expression

7. **Keep the full expression accessible** - the widget doesn't modify or lose any information

## Backward Compatibility

The original methods (`compute_minor()`, `compute_minor_fast()`, `compute_y_vector()`) still work exactly as before. The `*_display()` methods are completely optional and provide an alternative interface for better viewing in notebooks.

## Troubleshooting

### Widget not displaying?

Make sure you have ipywidgets installed and enabled:

```bash
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
```

For JupyterLab:
```bash
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

For Google Colab, ipywidgets should work out of the box.

### Browser freezing?

If clicking "Full LaTeX" freezes your browser, the expression is too large to render. Use the export feature instead:

```python
widget.export_to_file("expression.txt")
```

### Out of memory?

For very large systems, use:
- Lazy structure functions
- Disable simplification
- GPU acceleration (optional, see `GPU_ACCELERATION.md`)